In [38]:
import requests
import arcpy
import os
import zipfile

#### Define Variables
Define how many minutes the service area should cover.

In [1]:
minutes = 15

### Create Service Analysis Layers

#### Create the Service Analysis Layers and Add Random Locations

In [3]:
directionlist = ["FROM","TO"]
for direction in directionlist:
    #Make Service Area Analysis Layer
    arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb\MetroNetwork\MetroNetworkOG",
        layer_name=f"Service Area {direction} with 94",
        travel_mode="",
        travel_direction=f"{direction}_FACILITIES",
        cutoffs=[minutes],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS_AND_LINES",
        polygon_detail="STANDARD",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="5 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )

    #Make Service Area Analysis Layer
    arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork_Remove94.gdb\MetroNetwork94Remove\MetroNetwork94",
        layer_name=f"Service Area {direction} without 94",
        travel_mode="",
        travel_direction=f"{direction}_FACILITIES",
        cutoffs=[minutes],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS_AND_LINES",
        polygon_detail="STANDARD",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="5 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    
    #Add Locations to Service Area Layer
    arcpy.na.AddLocations(
        in_network_analysis_layer=f"Service Area {direction} with 94",
        sub_layer="Facilities",
        in_table=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\I94Points",
        field_mappings="Name Name #;CurbApproach # 0;Attr_Minutes # 0;Attr_Miles # 0;Breaks_Minutes # #;Breaks_Miles # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="MetroCenterlines SHAPE;MetroNetworkOG_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )

    arcpy.na.AddLocations(
        in_network_analysis_layer=f"Service Area {direction} without 94",
        sub_layer="Facilities",
        in_table=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\I94Points",
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Miles # 0;Breaks_Minutes # #;Breaks_Miles # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="Centerlines_Remove94 SHAPE;MetroNetwork94_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )

#### Solve the Service Area Analysis and Export (Be Prepared to Wait)

In [5]:
directionlist = ["FROM","TO"]
for direction in directionlist:
    arcpy.na.Solve(
        in_network_analysis_layer=f"Service Area {direction} with 94",
        ignore_invalids="SKIP",
        terminate_on_solve_error="TERMINATE",
        simplification_tolerance=None,
        overrides=""
    )
    #Export Service Area Polygons
    arcpy.conversion.ExportFeatures(
        in_features=fr"Service Area {direction} with 94\Polygons",
        out_features=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_94",
        where_clause="",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=r'FacilityID "FacilityID" true true true 4 Long 0 0,First,#,Service Area With 94\Polygons,FacilityID,-1,-1;Name "Name" true true true 1024 Text 0 0,First,#,Service Area With 94\Polygons,Name,0,1023;FromBreak "FromBreak" true true true 8 Double 0 0,First,#,Service Area With 94\Polygons,FromBreak,-1,-1;ToBreak "ToBreak" true true true 8 Double 0 0,First,#,Service Area With 94\Polygons,ToBreak,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Service Area With 94\Polygons,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,Service Area With 94\Polygons,Shape_Area,-1,-1',
        sort_field=None
    )
    #Export Service Line Features
    arcpy.analysis.PairwiseDissolve(
        in_features=fr"Service Area {direction} with 94\Lines",
        out_feature_class=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_94",
        dissolve_field="FacilityID",
        statistics_fields="Shape_Length SUM",
        multi_part="MULTI_PART",
        concatenation_separator=""
    )

    print(f"Exported Service Area {direction} with 94 Exisiting")

    arcpy.na.Solve(
        in_network_analysis_layer=f"Service Area {direction} without 94",
        ignore_invalids="SKIP",
        terminate_on_solve_error="TERMINATE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.conversion.ExportFeatures(
        in_features=fr"Service Area {direction} without 94\Polygons",
        out_features=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_Without94",
        where_clause="",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping=r'FacilityID "FacilityID" true true true 4 Long 0 0,First,#,Service Area With 94\Polygons,FacilityID,-1,-1;Name "Name" true true true 1024 Text 0 0,First,#,Service Area With 94\Polygons,Name,0,1023;FromBreak "FromBreak" true true true 8 Double 0 0,First,#,Service Area With 94\Polygons,FromBreak,-1,-1;ToBreak "ToBreak" true true true 8 Double 0 0,First,#,Service Area With 94\Polygons,ToBreak,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Service Area With 94\Polygons,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,Service Area With 94\Polygons,Shape_Area,-1,-1',
        sort_field=None
    )
    #Export Lines
    arcpy.analysis.PairwiseDissolve(
        in_features=fr"Service Area {direction} without 94\Lines",
        out_feature_class=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_Without94",
        dissolve_field="FacilityID",
        statistics_fields="Shape_Length SUM",
        multi_part="MULTI_PART",
        concatenation_separator=""
    )

    print(f"Exported Service Area {direction} Without 94 Exisiting")

Exported Service Area FROM with 94 Exisiting
Exported Service Area FROM Without 94 Exisiting
Exported Service Area TO with 94 Exisiting
Exported Service Area TO Without 94 Exisiting


### Compare Size of Polygons

#### Calculate Square Miles

In [6]:
directionlist = ["FROM","TO"]
for direction in directionlist:
    if isinstance(minutes, list):
        raise Exception("Minutes Cannot Be List to Calculate Size Difference")
    else:
        #Calulcate Area with 94 in Square Miles
        arcpy.management.CalculateField(
            in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_94",
            field="Area_With94",
            expression="!Shape_Area!  * 3.861E-7",
            expression_type="PYTHON3",
            code_block="",
            field_type="FLOAT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )
        #Calulate Line Length with 94 in Miles
        arcpy.management.CalculateField(
            in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_94",
            field="Length_With94",
            expression="!SUM_Shape_Length!  * 0.000621",
            expression_type="PYTHON3",
            code_block="",
            field_type="FLOAT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )
        #Calculate Area without 94 in Miles
        arcpy.management.CalculateField(
            in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_Without94",
            field="Area_Without94",
            expression="!Shape_Area!  * 3.861E-7",
            expression_type="PYTHON3",
            code_block="",
            field_type="FLOAT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )
        #Calculate Length of Lines without 94 in Miles
        arcpy.management.CalculateField(
            in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_Without94",
            field="Length_Without94",
            expression="!SUM_Shape_Length!  * 0.000621",
            expression_type="PYTHON3",
            code_block="",
            field_type="FLOAT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )

#### Join With and Without 94 Polygons/Line and Calculate Difference

In [7]:
directionlist = ["FROM","TO"]
for direction in directionlist:
    #Join Area with and without 94
    arcpy.management.JoinField(
        in_data=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_94",
        in_field="FacilityID",
        join_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_Without94",
        join_field="FacilityID",
        fields="Area_Without94",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )
    #Join length for with and without 94
    arcpy.management.JoinField(
        in_data=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_94",
        in_field="FacilityID",
        join_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_Without94",
        join_field="FacilityID",
        fields="Length_Without94",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )
    #Calculate Difference in Service Area
    arcpy.management.CalculateField(
        in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_94",
        field="ServiceDifference",
        expression="!Area_With94! - !Area_Without94!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    #Calculate Difference in Service Length
    arcpy.management.CalculateField(
        in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_94",
        field="ServiceDifference",
        expression="!Length_With94! - !Length_Without94!",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    #Calculate Percent Change for Area
    arcpy.management.CalculateField(
        in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_{direction}_94",
        field="PercentChange",
        expression="(!ServiceDifference! / !Area_With94!)*100",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

    #Calculate Percent Change for Length
    arcpy.management.CalculateField(
        in_table=fr"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceAreaLines_{direction}_94",
        field="PercentChange",
        expression="(!ServiceDifference! / !Length_With94!)*100",
        expression_type="PYTHON3",
        code_block="",
        field_type="FLOAT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )


In [ ]:
arcpy.analysis.Statistics(
    in_table=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_94",
    out_table=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_94_Statistics",
    statistics_fields="ServiceDifference MEAN;ServiceDifference STD;PercentChange MEAN;PercentChange STD",
    case_field=None,
    concatenation_separator=""
)


# List of fields to print
fields = ['MEAN_ServiceDifference', 'STD_ServiceDifference', 'Mean_PercentChange', 'STD_PercentChange']

# Use SearchCursor to iterate through the table and print selected fields
with arcpy.da.SearchCursor(r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\FinalProjectStart.gdb\ServiceArea_94_Statistics", fields) as cursor:
    for row in cursor:
        print(f"{fields}:{row}")
